# AlphaZero implementation for pulse sequence design
_Will Kaufman, December 2020_

[Dalgaard et. al. (2020)](https://www.nature.com/articles/s41534-019-0241-0) applied this approach to constructing shaped pulses (as I understand it), but in theory this should be as applicable to pulse sequence design, if not more so. The original [AlphaZero paper](https://science.sciencemag.org/content/362/6419/1140.full) is here.

The general idea behind AlphaZero (as I understand it) is to do a "smart" tree search that balances previous knowledge (the policy), curiosity in unexplored branches, and high-value branches. My thought is that this can be improved with AHT (i.e. knowing that by the end of the pulse sequence, the pulse sequence must be cyclic (the overall frame transformation must be identity) and there must be equal times spent on each axis). This will provide a hard constraint that will (hopefully) speed up search.

In [1]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
import sys, os
from random import sample

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
sys.path.append(os.path.abspath('..'))
import pulse_sequences as ps
import alpha_zero as az

In [253]:
import importlib
importlib.reload(az)
importlib.reload(ps)

<module 'pulse_sequences' from '/Users/willkaufman/Projects/rl_pulse/rl_pulse/pulse_sequences.py'>

## Define the spin system

In [3]:
delay = 1e-2  # time is relative to chemical shift strength
pulse_width = 5e-3
N = 3  # number of spins
ensemble_size = 5

In [4]:
X, Y, Z = ps.get_collective_spin(N)

In [5]:
Hsys_ensemble = [ps.get_Hsys(N) for _ in range(ensemble_size)]
pulses_ensemble = [
    ps.get_pulses(H, X, Y, Z, pulse_width, delay, rot_error=0.01) for H in Hsys_ensemble
]

In [6]:
Utarget = qt.identity(Hsys_ensemble[0].dims[0])

## Average Hamiltonian theory

To keep track of the average Hamiltonian (to lowest order), I'm defining a frame matrix and applying rotation matrices to the frame matrix, then determining how $I_z$ transforms during the pulse sequence. The last row in the frame matrix corresponds to the current transformed value of $I_z$.

In [7]:
ps.count_axes(ps.yxx48)

[8, 8, 8, 8, 8, 8]

In [8]:
ps.get_valid_time_suspension_pulses([0,1,1,], len(ps.pulse_names), 6)

[1, 3, 4]

## Tree search

Define nodes that can be used for tree search, with additional constraints that the lowest-order average Hamiltonian matches the desired Hamiltonian.

(deleted code that implemented tree search with constraints, see GitHub repo commits on 12/8 for code)

For 12-pulse sequences, calculated 16 branches at depth 4 in a minute, so about 1 every 4 seconds. At depth 4 there are $5^4 = 625$ branches, so that'll take $4 * 625 = 41$ hours to fully run. Alternatively, you can generate random pulse sequences until there's one that has the proper lowest-order average and cyclic property.

## Smarter search with MCTS

Following the [supplementary materials description under "Search"](https://science.sciencemag.org/content/sci/suppl/2018/12/05/362.6419.1140.DC1/aar6404-Silver-SM.pdf) to do rollouts and backpropagate information.

In [9]:
config = az.Config()
config.num_simulations = 500
ps_config = ps.PulseSequenceConfig(N, ensemble_size, pulse_width, delay, 6, Utarget)

In [68]:
# %load_ext snakeviz
# %snakeviz -t az.make_sequence(config, ps_config, None)

In [467]:
# stats = az.make_sequence(config, ps_config, None)

In [468]:
# ps_config.value()

In [469]:
# stats

## Replay buffer

Inspired by [this pytorch tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html).

In [257]:
rb = az.ReplayBuffer(100000)

In [258]:
config = az.Config()
config.num_simulations = 100

For the purposes of saving data in a reasonable way (and using RNN), the state is represented by a sequence, where 0 indicates the start of sequence, and 1-5 are the possible pulses (1: delay, 2: x, etc...).

In [259]:
for _ in range(5):
    print(f'creating pulse sequence {_}')
    ps_config = ps.PulseSequenceConfig(N, ensemble_size, pulse_width, delay, 48, Utarget)
    stats = az.make_sequence(config, ps_config, None)
    for s in stats:
        state = az.one_hot_encode(s[0])
        probs = torch.Tensor(s[1])
        value = torch.Tensor([s[2]])
        rb.add((state,
                probs,
                value))

creating pulse sequence 0
creating pulse sequence 1
creating pulse sequence 2
creating pulse sequence 3
creating pulse sequence 4


In [260]:
len(rb)

240

In [33]:
# rb.sample()

# TEMP

In [299]:
sequence = torch.randint(5, (48,))

In [307]:
states = [az.one_hot_encode(sequence[:i].float()) for i in range(48)]

In [312]:
a = F.softmax(torch.randn(30,5), 1)

## Neural networks for policy, value estimation

Batched tensors have shape `B * T * ...` where `B` is batch size and `T` is the timestep. Different from default behavior, but more intuitive to me.

In [261]:
p = az.Policy()

In [262]:
batch_size = 30
seq_length = 48

In [263]:
minibatch = rb.sample(batch_size)

In [264]:
states = [i[0] for i in minibatch]

In [267]:
packed_states = az.pad_and_pack(states)

In [268]:
output, (h, c) = p(packed_states)

In [ ]:
# output

## Optimize policy based on target distribution

In [277]:
policy_optimizer = optim.Adam(p.parameters())

In [278]:
minibatch = rb.sample(batch_size)
states = [i[0] for i in minibatch]
probs = torch.cat([i[1].unsqueeze(0) for i in minibatch])
packed_states = pad_and_pack(states)

In [279]:
for _ in range(1000):
    outputs, __ = p(packed_states)
    loss = -1 / len(states) * torch.sum(probs * torch.log(outputs))
    if _ % 100 == 0:
        print(loss)
    loss.backward()
    policy_optimizer.step()

tensor(1.6102, grad_fn=<MulBackward0>)


In [282]:
p(states[3].unsqueeze(0))[0]

tensor([[0.2491, 0.1775, 0.1753, 0.2160, 0.1821]], grad_fn=<SoftmaxBackward>)

In [283]:
probs[3]

tensor([0.2200, 0.3300, 0.0000, 0.2200, 0.2300])

## Value network

## TODO

- [ ] Value network (below)
- [ ] Bring it all together with MCTS...
- [ ] Set up Discovery environment
- [ ] Run it and (hopefully) rejoice!

In [284]:
v = az.Value()

## Optimize value function

In [288]:
value_optimizer = optim.Adam(v.parameters())

In [289]:
minibatch = rb.sample(batch_size)
states = [i[0] for i in minibatch]
values = torch.cat([i[2].unsqueeze(0) for i in minibatch])
packed_states = pad_and_pack(states)

In [290]:
for _ in range(1000):
    outputs, __ = v(packed_states)
    loss = F.mse_loss(v(packed_states)[0], values)
    if _ % 100 == 0:
        print(loss)
    loss.backward()
    value_optimizer.step()

tensor(0.0867, grad_fn=<MseLossBackward>)
tensor(0.0005, grad_fn=<MseLossBackward>)
tensor(0.0048, grad_fn=<MseLossBackward>)
tensor(0.0104, grad_fn=<MseLossBackward>)
tensor(0.0013, grad_fn=<MseLossBackward>)
tensor(0.0102, grad_fn=<MseLossBackward>)
tensor(0.0012, grad_fn=<MseLossBackward>)
tensor(0.0085, grad_fn=<MseLossBackward>)
tensor(0.0016, grad_fn=<MseLossBackward>)
tensor(0.0127, grad_fn=<MseLossBackward>)


In [291]:
v(states[7].unsqueeze(0))[0]

tensor([[0.0736]], grad_fn=<AddmmBackward>)

In [292]:
values[7]

tensor([0.0513])